In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
from pymongo import MongoClient
from tqdm import tqdm_notebook as tqdm
from nltk.tokenize import word_tokenize
import gensim
from pprint import pprint
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import spacy
from nltk.corpus import stopwords

In [30]:
client = MongoClient('localhost',27017)

In [31]:
db = client['covid']
studies = db.studies 

In [4]:
doc_path = '/Users/jeloretizo/Library/Mobile Documents/com~apple~CloudDocs/CORD-19-research-challenge'

In [5]:
metadata = pd.read_csv(doc_path + '/metadata.csv')

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
0,xqhn0vbp,1e1286db212100993d03cc22374b624f7caee956,PMC,Airborne rhinovirus detection and effect of ul...,10.1186/1471-2458-3-5,PMC140314,12525263.0,no-cc,"BACKGROUND: Rhinovirus, the most common cause ...",2003-01-13,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",BMC Public Health,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
1,gi6uaa83,8ae137c8da1607b3a8e4c946c07ca8bda67f88ac,PMC,Discovering human history from stomach bacteria,10.1186/gb-2003-4-5-213,PMC156578,12734001.0,no-cc,Recent analyses of human pathogens have reveal...,2003-04-28,"Disotell, Todd R",Genome Biol,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
2,le0ogx1s,NaN,PMC,A new recruit for the army of the men of death,10.1186/gb-2003-4-7-113,PMC193621,12844350.0,no-cc,"The army of the men of death, in John Bunyan's...",2003-06-27,"Petsko, Gregory A",Genome Biol,NaN,NaN,False,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,fy4w7xz8,0104f6ceccf92ae8567a0102f89cbb976969a774,PMC,Association of HLA class I with severe acute r...,10.1186/1471-2350-4-9,PMC212558,12969506.0,no-cc,BACKGROUND: The human leukocyte antigen (HLA) ...,2003-09-12,"Lin, Marie; Tseng, Hsiang-Kuang; Trejaut, Jean...",BMC Med Genet,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...
4,0qaoam29,5b68a553a7cbbea13472721cd1ad617d42b40c26,PMC,A double epidemic model for the SARS propagation,10.1186/1471-2334-3-19,PMC222908,12964944.0,no-cc,BACKGROUND: An epidemic of a Severe Acute Resp...,2003-09-10,"Ng, Tuen Wai; Turinici, Gabriel; Danchin, Antoine",BMC Infect Dis,NaN,NaN,True,True,custom_license,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...


In [115]:
metadata.loc[metadata['sha'] == 'ff766484e50701bbb36ed45d3a4464d817a471ff']['abstract'].iloc[0]

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
40896,nl37p9vp,ff766484e50701bbb36ed45d3a4464d817a471ff,Elsevier,Middle East Respiratory Syndrome Interpreted: ...,10.1016/j.amjmed.2016.04.030,PMC7124269,27215907.0,els-covid,NaN,2016-09-30,"Lau, Susanna K.P.; Chan, Jasper F.W.; Hung, Iv...",The American Journal of Medicine,NaN,NaN,True,False,custom_license,https://doi.org/10.1016/j.amjmed.2016.04.030


In [70]:
comm_pmc_json = doc_path + '/comm_use_subset/comm_use_subset/pmc_json/'
comm_pdf_json = doc_path + '/comm_use_subset/comm_use_subset/pdf_json/'
custom_pmc_json = doc_path + '/custom_license/custom_license/pmc_json/'
custom_pdf_json = doc_path + '/custom_license/custom_license/pdf_json/'

In [71]:
comm_pmc_json_files = [f for f in listdir(comm_pmc_json) if isfile(join(comm_pmc_json, f))]
comm_pdf_json_files = [f for f in listdir(comm_pdf_json) if isfile(join(comm_pdf_json, f))]
custom_pmc_json_files = [f for f in listdir(custom_pmc_json) if isfile(join(custom_pmc_json, f))]
custom_pdf_json_files = [f for f in listdir(custom_pdf_json) if isfile(join(custom_pdf_json, f))]

In [8]:
len(comm_pdf_json_files)

9524

In [72]:
len(comm_pmc_json_files)

9148

In [74]:
# comm_pmc_json_files

In [10]:
len(custom_pdf_json_files)

26505

In [11]:
len(custom_pmc_json_files)

7797

In [ ]:
def find_abstract(research_id):
    ''' Helper function to aggregate abstract and metadata '''
    if research_id is not None:
        if 'PMC' in research_id:
            try:
                abstract = metadata.loc[metadata['pmcid'] == research_id]['abstract'].iloc[0]
            except:
                abstract = None
        else:
            try: 
                abstract = metadata.loc[metadata['sha'] == research_id]['abstract'].iloc[0]
            except:
                abstract = None
    else:
        abstract = None
    return abstract

In [82]:
def load_files(files_list, files_path):
    for orig_file in tqdm(files_list):
        with open((files_path + orig_file), 'r') as file:
            try:
                file = json.load(file)
    #             print(custom_pdf_json_file)
            except:
                print('Warning! File not read:', orig_file)
            res = {}
            try:
                res['paper_id'] = file['paper_id'] 
            except:
                res['paper_id'] = None

            try:
                res['abstract'] = file['abstract'][0]['text']
            except:
                res['abstract'] = None

            try: 
                res['title'] = file['metadata']['title']
            except:
                res['title'] = None

            try:
                res['authors'] = file['metadata']['authors']
            except:
                res['authors'] = None

            try:
                res['other_metadata'] = file['metadata']
            except:
                res['other_metadata'] = None

            try:
                res['other_abstract'] = file['abstract']
            except:
                res['other_abstract'] = None

    #         data = {}
            full_text = ''
            try:
    #             for i in file['body_text']:
    #                 section = i['section'].lower()
    #                 if '.' in section:
    #                     section = section.replace('.', ' ')
    #                 if section in list(data.keys()): 
    #                     new_text = data[section] + "\n" + i['text']
    #                     data[section] = new_text
    #     #                 new_text
    #     #                 data[i['section']].append(i['text'])
    #             #         print('a')
    #                 else:
    #                     data[section] = i['text']
                for i in file['body_text']:
                    full_text += i['text']

            except:
                full_text = None

            try:
                res['body'] = full_text
            except:
                res['body'] = None
            try:
                studies.insert_one(res, bypass_document_validation=True)
            except:
                print('Warning! File not inserted:', orig_file)

In [83]:
load_files(custom_pdf_json_files, custom_pdf_json)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Warning! File not read: ffc89f2bd7f513dae147b4bd0043164db16e8dc0.json
Warning! File not read: ff966b5218e3c9433966d622a806710b820a60d6.json
Warning! File not read: ff9398e9978220ab54cd85a98920d90b05c190d6.json
Warning! File not read: ffb877d67fd30e14677580c292ca9caf64659987.json
Warning! File not read: ff6c4e420ae293c8130a3a71a1ab9e466ccf0b40.json
Warning! File not read: ff765aa3a876d263d37993712a503bc2a9f5ef0c.json
Warning! File not read: ff740fd015f63477d7b236d0ffee2b7639b6dcb0.json
Warning! File not read: ffc93dea5a5f4e6e65ad155e87aafa98aa8c54cf.json
Warning! File not read: ff80b3d9f398a7e1b51c1c2dc6fdbc97969d24f0.json
Warning! File not read: ffd3606ed51d3129db5863b2246ec587e07323c8.json
Warning! File not read: ffb77680c0f79491003331d1e3d9dde3c4fcdfd9.json
Warning! File not read: ffab348b231771ee27738a4426a87a6a04f3994f.json
Warning! File not read: ff7d49ac4008f60ef9c5a437e0d504dcefd1246f.json
Warning! File not read: ffd73c393641618860486e897197311f630bf67f.json
Warning! File not re

In [84]:
load_files(custom_pmc_json_files, custom_pmc_json)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Warning! File not read: PMC1383626.xml.json
Warning! File not read: PMC1850917.xml.json
Warning! File not read: PMC1635791.xml.json
Warning! File not read: PMC1160239.xml.json
Warning! File not read: PMC1518840.xml.json
Warning! File not read: PMC1182165.xml.json
Warning! File not read: PMC2259160.xml.json
Warning! File not read: PMC2206501.xml.json
Warning! File not read: PMC1592674.xml.json
Warning! File not read: PMC1566363.xml.json
Warning! File not read: PMC1769406.xml.json
Warning! File not read: PMC2435248.xml.json
Warning! File not read: PMC1474450.xml.json
Warning! File not read: PMC1278949.xml.json
Warning! File not read: PMC1074749.xml.json
Warning! File not read: PMC2589215.xml.json
Warning! File not read: PMC1821001.xml.json
Warning! File not read: PMC2516372.xml.json
Warning! File not read: PMC156578.xml.json
Warning! File not read: PMC1519403.xml.json
Warning! File not read: PMC1602263.xml.json
Warning! File not read: PMC2443186.xml.json
Warning! File not read: PMC182099

In [84]:
for i in file.keys():
    print(i)

paper_id
metadata
abstract
body_text
bib_entries
ref_entries
back_matter


In [85]:
file['paper_id']

'86a998617c077f4fe2ab26214995a3548fbc0fc5'

In [97]:
full_data = pd.DataFrame(list(studies.find(
    {}, {'_id': 0, 'authors': 0, 'other_metadata': 0, 'other_abstract': 0})))
full_data.drop('abstract', axis=1).to_csv('./all_research.csv', index=False)

In [2]:
full_data = pd.read_csv('./all_research.csv')

In [3]:
full_data = full_data.drop_duplicates(['title', 'new_abstract'], keep='first')

In [4]:
full_data = full_data.dropna()

In [5]:
subset = full_data.head(5000)

In [6]:
subset['lower_body'] = subset['body'].str.lower()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
subset['lower_body_tokens'] = subset['lower_body'].apply(lambda x: word_tokenize(str(x)))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
subset['lower_body_tokens'] = subset['lower_body_tokens'].apply(lambda x: [w for w in x if w.isalpha()])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
stop_words = stopwords.words('english')
def remove_stopwords(word_tokens):
    return [w for w in word_tokens if not w in stop_words] 

In [19]:
subset['lower_body_tokens'] = subset['lower_body_tokens'].apply(lambda x: remove_stopwords(x))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
all_body = list(subset['lower_body_tokens'])

In [21]:
model = gensim.models.Word2Vec(all_body, size=100, window=5, min_count=1)

In [22]:
word_vectors = model.wv

In [25]:
model.most_similar(positive='coronavirus')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('cov', 0.8944141268730164),
 ('norovirus', 0.8365249633789062),
 ('picornavirus', 0.8242976665496826),
 ('adenovirus', 0.8122658729553223),
 ('paramyxovirus', 0.8113682866096497),
 ('coxsackievirus', 0.8097898960113525),
 ('enterovirus', 0.809447169303894),
 ('flavivirus', 0.8092314004898071),
 ('herpesvirus', 0.8069195747375488),
 ('retroviruses', 0.8040987849235535)]

In [26]:
model.most_similar(positive='transmission')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('spread', 0.8131103515625),
 ('contact', 0.7746533155441284),
 ('transmissibility', 0.7723363637924194),
 ('spillover', 0.7054035663604736),
 ('epidemic', 0.7000199556350708),
 ('transmissions', 0.6984718441963196),
 ('airborne', 0.6930473446846008),
 ('emergence', 0.68519526720047),
 ('epidemics', 0.6673940420150757),
 ('spreads', 0.6639245748519897)]

In [95]:
search_string = 'What do we know about the transmission of the virus?'

In [93]:
from rake_nltk import Rake
r = Rake()

In [94]:
r.extract_keywords_from_text(search_string)
r.get_ranked_phrases() 

['virus', 'transmission', 'know']

In [81]:
new_search = ''
for i in r.get_ranked_phrases():
    new_search += ' ' + i

In [83]:
new_search.strip()

'know covid'

In [53]:
for i in subset.iterrows():
    print(i[1]['paper_id'])

...


8134e0946079f5ef0bbbd7b65ce13657483b8160
a08f5fd1ac9fc3e33a771787d584a845a8558cae
4e388424e65f63b5cc6b567b0980d430cb2d738c
6224ce8bb65afbb10b08238d5ecde8991d7ab0e2
1f1bfc20d5fa1896482283874f8b7971e51426ec
842863c4f79b4d13ef260ef46b9882e8afe2b8ee
b35188139844b656464c0727fa39ec03d970c443
513f57a7efa430c143b840c17beb62c010429aac


In [96]:
distances = [(i[1]['paper_id'], 
              model.wmdistance(search_string, 
                               i[1]['body'])) for i in full_data.iterrows()]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
class research_model:
    def search(search_string):
        

In [97]:
sorted(distances,key=lambda x: x[1])[:5]

[('3a8471a27ef5de09200e4d659467e203c900ae48', 2.3013047687998696),
 ('PMC7123587', 2.3067632244057754),
 ('b24beffd1af8f836c7931d9729b74078feb7e5ba', 2.384921039897632),
 ('PMC7121202', 2.4490523670813626),
 ('PMC7123263', 2.5113062362673215)]

In [104]:
full_data.loc[full_data['paper_id'] == 'PMC7121202']['title'].iloc[0]

'A Locally Transmitted Case of SARS-CoV-2 Infection in Taiwan'

In [105]:
full_data.loc[full_data['paper_id'] == 'PMC7121202']['new_abstract'].iloc[0]

'On January 25, 2020, a 52-year-old woman with a history of type 2 diabetes presented with fever to an emergency department in central Taiwan. She was admitted to the hospital because of suspicion of pneumonia associated with SARS-CoV-2 infection. She had lived in Wuhan from October 21, 2019, to January 20, 2020. She returned to Taiwan from Wuhan on January 20 on an airplane. On the same day, a throat swab was obtained from another passenger on that flight; that passenger was confirmed to have the first known imported case of SARS-CoV-2 infection in Taiwan when the swab was found to be positive for the virus on January 21.'

In [91]:
full_data.loc[full_data['paper_id'] == 'a6f478369adbfd8be967251a9bb0dd17fa436c45']['body']

9394    www.lungca.org 4 5 6 7 8 9 10 11 12 13 14 15 1...
Name: body, dtype: object

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')

In [7]:
doc_term_matrix = count_vect.fit_transform(subset['body'].values.astype('U'))

In [8]:
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.partial_fit(doc_term_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [9]:
doc_term_matrix

<5000x80491 sparse matrix of type '<class 'numpy.int64'>'
	with 4602534 stored elements in Compressed Sparse Row format>

In [18]:
LDA

array([ 6661, 16776,  6161, ..., 31054, 11235,  2840])

In [10]:
first_topic =LDA.components_[0]

In [11]:
for i in first_topic.argsort()[-20:]:
    print(count_vect.get_feature_names()[i])

epidemic
human
countries
control
research
infectious
surveillance
diseases
case
outbreak
influenza
information
population
public
risk
transmission
model
cases
disease
health


In [12]:
count_vect.get_feature_names()[i]

'health'

In [13]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['influenza', 'information', 'population', 'public', 'risk', 'transmission', 'model', 'cases', 'disease', 'health']


Top 10 words for topic #1:
['infections', 'positive', 'viral', 'children', 'virus', 'samples', 'clinical', 'influenza', 'respiratory', 'patients']


Top 10 words for topic #2:
['genome', 'virus', 'viral', 'viruses', 'gene', 'sequence', 'sequences', 'genes', 'et', 'al']


Top 10 words for topic #3:
['activity', 'rna', 'figure', 'proteins', 'virus', 'viral', 'expression', 'protein', 'cell', 'cells']


Top 10 words for topic #4:
['antibodies', 'viral', 'protein', 'antibody', 'ml', 'infected', 'cell', 'mice', 'virus', 'cells']




In [14]:
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

(5000, 5)

In [15]:
subset['Topic'] = topic_values.argmax(axis=1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
subset['Topic'].unique()

array([4, 3, 2, 1, 0])

In [17]:
subset.head()

,paper_id,title,body,new_abstract,Topic
0,5e0c586f047ff909c8ed3fe171c8975a90608d08,Neutralizing antibodies against porcine epidem...,"Porcine epidemic diarrhea virus (PEDV), which ...",BACKGROUND: Porcine epidemic diarrhea virus (P...,4
1,1579fbff7af9b156c6f49fee0526e48f852ea460,A Recombinant Newcastle Disease Virus (NDV) Ex...,"Generation of rNDVs expressing S1, S2 or S pro...",Infectious bronchitis virus (IBV) causes a hig...,4
3,38aa050ad79d8a1d7022c33535255ce9d47914e5,Potent Inhibition of Junín Virus Infection by ...,Arenaviruses are enveloped RNA viruses with bi...,The new world arenavirus Junín virus (JUNV) is...,3
4,61722c462b054f36461375e96e502cbf22648c04,2 convergent Research center for emerging Viru...,"In this study, the anti-dengue activity of nic...",Dengue fever is one of the most important mosq...,3
5,7107f088cbed45d8a06a026276ccf4d602d50f10,Microglia Play a Major Role in Direct Viral-In...,Microglia are specialized macrophages of the C...,Microglia are the resident macrophage-like pop...,4
